In [1]:
import parcels
from parcels import Field,FieldSet,ParticleSet,Variable,JITParticle,AdvectionRK4,ErrorCode,BrownianMotion2D
import datetime
from datetime import timedelta as delta 
from glob import glob
import math as m
import xarray as xr
import numpy as np
from operator import attrgetter

from netCDF4 import Dataset
from scipy.interpolate import griddata, Rbf
from parcels import rng as random

In [12]:
##Function and Kernels

class PlasticParticle(JITParticle):  
    # Define a new particle class that contains three extra variables
    #distance = Variable('distance', initial=0., dtype=np.float32)     #the distance travelled
    #prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
    #                    initial=attrgetter('lon'))                    #the previous longitude
    #prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
    #                    initial=attrgetter('lat'))                    #the previous latitude
    u = Variable('u', dtype = np.float32)
    v = Variable('v', dtype = np.float32)
    beached = Variable('beached', dtype =np.float32, initial = 0 )
    Kym1 = Variable('Kh_meridional', dtype = np.float32)
    Kxm1 = Variable('Kh_zonal', dtype = np.float32)

def Beached(particle, fieldset, time):
    (u, v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
    
    if fabs(u) < 1e-18 and fabs(v) < 1e-18: #u,v never really = 0, instead use a really small number
        particle.beached = 1                #1 = beached, 0 = sea
    
def TotalDistance(particle, fieldset, time): 
    #Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    #Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    #Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat

def Diffusion(particle, fiedlset, time):
    if particle.beached == 0 :
        #dWx = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)
        #dWy = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)

        #bx = math.sqrt(2 * fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon])
        #by = math.sqrt(2 * fieldset.Kh_meridional[time, particle.depth, particle.lat, particle.lon])

        #particle.lon += bx * dWx
        #particle.lat += by * dWy
        
        #Parcels implemented kernel (AdvectionRK4DiffusionM1):
        # RK4 terms
        (u1, v1) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        lon1, lat1 = (particle.lon + u1 * .5 * particle.dt, particle.lat + v1 * .5 * particle.dt)
        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1]
        lon2, lat2 = (particle.lon + u2 * .5 * particle.dt, particle.lat + v2 * .5 * particle.dt)
        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2]
        lon3, lat3 = (particle.lon + u3 * particle.dt, particle.lat + v3 * particle.dt)
        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3]

        # Wiener increment with zero mean and std of sqrt(dt)
        dWx = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)
        dWy = random.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3)

        Kxp1 = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon + fieldset.dres]
        Kxm1 = fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon - fieldset.dres]
        dKdx = (Kxp1 - Kxm1) / (2 * fieldset.dres)
        bx = math.sqrt(2 * fieldset.Kh_zonal[time, particle.depth, particle.lat, particle.lon])

        Kyp1 = fieldset.Kh_meridional[time, particle.depth, particle.lat + fieldset.dres, particle.lon]
        Kym1 = fieldset.Kh_meridional[time, particle.depth, particle.lat - fieldset.dres, particle.lon]
        dKdy = (Kyp1 - Kym1) / (2 * fieldset.dres)
        by = math.sqrt(2 * fieldset.Kh_meridional[time, particle.depth, particle.lat, particle.lon])

        # Particle positions are updated only after evaluating all terms.
        particle.lon += ((u1 + 2 * u2 + 2 * u3 + u4) / 6.) * particle.dt + 0.5 * dKdx * (dWx**2 + particle.dt) + bx * dWx
        particle.lat += ((v1 + 2 * v2 + 2 * v3 + v4) / 6.) * particle.dt + 0.5 * dKdy * (dWy**2 + particle.dt) + by * dWy

def periodicBC(particle, fieldset, time):
    """
    Kernel for periodic values in longitude
    """
    if particle.lon < 0.:
        particle.lon += 360.
    elif particle.lon >= 360.:
        particle.lon -= 360.
        
def OutOfBounds(particle, fieldset, time):
    particle.delete()
    
def set_currents (uvfiles):
    #all in the same file
    filenames = {'U': uvfiles,'V': uvfiles}
    dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
    variables = {'U':'uo', 'V' : 'vo'}
    uvindices = {'lon': range(2484, 2664), 'lat': range(1452, 1536)} #define domain 
   
    
    fset_currents = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation = False, 
                                    indices = uvindices)
    fset_currents.add_periodic_halo(zonal=True, meridional=False, halosize=5)
    
    return fset_currents


def set_stokes(stokesfiles):
    stokesfilenames = {'U': stokesfiles,'V': stokesfiles}
    stokesdimensions = {'lon': 'longitude','lat': 'latitude', 'time': 'time'}
    stokesvariables ={'U': 'uuss', 'V': 'uuss'}
    
    stokesindices = {'lon': range(420, 440), 'lat': range(237, 253)} #define domain    
    

    fset_stokes = FieldSet.from_netcdf(stokesfiles,stokesvariables,stokesdimensions,
                                       allow_time_extrapolation = False, indices = stokesindices)
    fset_stokes.add_periodic_halo(zonal=True, meridional=False, halosize=5)

    
    return fset_stokes

def particles_grid(name, spacing = 0.2, minlon = 27, maxlon = 42, minlat = 41.5, maxlat = 47):
    #Uniform grid of particles
    
    griddir = r'/data/oceanparcels/input_data/NEMO-MEDUSA/ORCA0083-N006/domain/bathymetry_ORCA12_V3.3.nc'
    outdir= '/science-nfs-sys/vsm01/users/6312454/' + 'BS_grid'
    
    filename = griddir
    data = Dataset(filename,'r')
    bathy=np.array(data['Bathymetry'])
    lon=np.array([data['nav_lon']][0])
    lat=np.array([data['nav_lat']][0])
    
    grid=np.mgrid[minlon:maxlon:spacing,minlat:maxlat:spacing]
    n=grid[0].size;
    lons=np.reshape(grid[0],n)
    lats=np.reshape(grid[1],n)
      
    points = np.array([lon.flatten(), lat.flatten()]).T
    values = bathy.flatten()
    xi = (lons, lats)
    
    bathy_p = griddata(points, values, xi, method='nearest')
    
    
    Lons = np.array([lons[i] for i in range(len(lons)) if bathy_p[i] >17])
    Lats = np.array([lats[i] for i in range(len(lats)) if bathy_p[i] >17])
    
    Lons[Lons<0.] += 360.
    np.save(outdir + '_Longitude_' + str(name),Lons)
    np.save(outdir + '_Latitude_' + str(name),Lats)

    return Lons, Lats


In [ ]:
## Load data
datadir = '/data/oceanparcels/input_data'

stokesfiles = sorted(glob(datadir+'/WaveWatch3data/CFSR/WW3-GLOB-30M_*.nc'))
uvfiles = sorted(glob(datadir +'/CMEMS/GLOBAL_REANALYSIS_PHY_001_030/mercatorglorys12v1_gl12_mean_*.nc'))

##Coordinates particles grid
name = '05'
lons,lats = particles_grid(name)

## Fiedlset 
currents = set_currents(uvfiles)
stokes = set_stokes(stokesfiles)

fset = FieldSet(U = currents.U, V = currents.V)
fset_stokes = FieldSet(U =currents.U + stokes.U, V = currents.V + stokes.V)

In [11]:
fset.Kh = 0.25
fset_stokes.Kh = 0.25

In [ ]:
#Add field Kh

size2D = (fset.U.grid.ydim, fset.U.grid.xdim)

fset.add_field(Field('Kh_zonal', data=10* np.ones(size2D),lat=fset.U.grid.lat,lon=fset.U.grid.lon,
                     mesh='spherical', allow_time_extrapolation=True))
fset.add_field(Field('Kh_meridional', data=10 * np.ones(size2D),lon=fset.U.grid.lon, lat=fset.U.grid.lat,
                     mesh='spherical', allow_time_extrapolation=True))

In [4]:
#Add field for BM2D

size2D = (fset.U.grid.ydim, fset.U.grid.xdim)
KH_z = 0.25
alpha = 1
L = fset.U.grid.ydim
beta = np.zeros(fset.U.grid.ydim)
y_K = fset.V.grid.lat

for yi in range(len(y_K)):
    if y_K[yi] < L/2:
        beta[yi] = y_K[yi]*np.power(L - 2*y_K[yi], 1/alpha)
    elif y_K[yi] >= L/2:
        beta[yi] = (L - y_K[yi])*np.power(2*y_K[yi] - L, 1/alpha)

KH_m = KH_z*(2*(1+alpha)*(1+2*alpha))/(alpha**2*np.power(L, 1+1/alpha))*beta
Kh_meridional = np.array([KH_m for x in range(190)]).T


fset.add_field(Field('Kh_zonal', data=KH_z*np.ones(size2D),lat=fset.U.grid.lat,lon=fset.U.grid.lon,
                     mesh='spherical', allow_time_extrapolation=True))
fset.add_field(Field('Kh_meridional', data=Kh_meridional*np.ones(size2D), 
                     #data=8 * np.ones(size2D),
                     lon=fset.U.grid.lon,lat=fset.U.grid.lat,mesh='spherical', allow_time_extrapolation=True))

fset.add_constant('dres', 0.00002)

fset_stokes.add_field(Field('Kh_zonal', data=KH_z*np.ones(size2D),lat=fset.U.grid.lat,lon=fset.U.grid.lon,
                     mesh='spherical', allow_time_extrapolation=True))
fset_stokes.add_field(Field('Kh_meridional', data=Kh_meridional*np.ones(size2D), 
                            #data=8 * np.ones(size2D),
                            lon=fset.U.grid.lon,lat=fset.U.grid.lat,mesh='spherical', 
                            allow_time_extrapolation=True))

fset_stokes.add_constant('dres', 0.00002)

In [14]:
###RUN 4

startlats = lats
startlons = lons
startlons = [l + 360 for l in startlons]


rundays = 30
dt = 30         #[min]
nperloc = 30

starttime = datetime.datetime(2015,1,1,0,0)
starttime = np.repeat(starttime, len(startlats))

repeatdt = delta(days=30)



psetname ='RUN_4testDiffusion'

name = str(psetname)+str(rundays)+'d_'+str(dt)+'m'   #outputfile name


pset_RUN4 = ParticleSet(fieldset=fset_stokes, pclass=PlasticParticle, lon=np.repeat(startlons, [nperloc]),
                        lat=np.repeat(startlats, [nperloc]), time=np.repeat(starttime, [nperloc])
                        #,repeatdt = repeatdt
                       )

kernels = pset_RUN4.Kernel(AdvectionRK4) + Beached + periodicBC + Diffusion +Speed + Kh

pset_RUN4.execute(kernels, runtime = delta(days = rundays), dt = delta(minutes= dt),
                    output_file = pset_RUN4.ParticleFile(name = name, outputdt=delta(hours=12)), 
                    recovery = {ErrorCode.ErrorOutOfBounds: OutOfBounds},verbose_progress =False)


AttributeError: 'FieldSet' object has no attribute 'Kh_zonal'